In [ ]:
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adadelta(),metrics=['accuracy'])

hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist1.h5')
print("Saving the model as mnist1.h5")



In [ ]:
pip install keras

In [8]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, ImageOps, Image
import numpy as np
from numpy import asarray
import cv2


model = load_model('mnist.h5')
dg=0
    

def divide(img1,x,j):
    img = cv2.imread('temp.jpg')
    a_part = img[:, x:j] 
  
    # [:,half:] means al the rows and all
    # the columns from index half to the end
    # cv2.imshow is used for displaying the image
    #cv2.imshow('a', a_part)
    cv2.imwrite('output.jpg',a_part)
    

    image = Image.open('back.jpg')
    logo = Image.open('output.jpg')
    image_copy = image.copy()
    position = ((image_copy.width//2 - logo.width//2), (image_copy.height//2 - logo.height//2))
    image_copy.paste(logo, position)
    image_copy.save('output.jpg')
    img2  = Image.open("output.jpg")
    predict_digit(img2)

def coordinate(img1):
        img = img1.convert('1')
        #img.show()
        # convert image to numpy array
        data = asarray(img)
        #print(type(data))
        # summarize shape
        #(data)
        x=0
        for j in range (0,img1.width):
            a=0
            for i in range(0,img1.height):
                if data[i][j] == False:
                    a=1
            if a==0:
                if(j!=x):
                    #print(j)
                    #print(x)
                    divide(img1,x,j)
                x=j+1

       

def predict_digit(img):
    global dg
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = ImageOps.invert(img)
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    
    #predicting the class
    res = model.predict([img])[0]
    dg=dg*(10) + np.argmax(res)
    

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=800, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        self.geometry("1200x350+0+0")
        self.resizable(False, False)
        
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
            
    def classify_handwriting(self):
        global dg
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect
        rect=(a+8,b+8,c+200,d+80)
        im = ImageGrab.grab(rect)
        #im.show()
        im.save("temp.jpg")
        coordinate(im)
        #self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
        self.label.configure(text= str(dg))
        dg=0
        d=0
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=4
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
        
    
       
app = App()


mainloop()
